# Analysis for the opening of a Gastropub in the city of Quito, Ecuador
## By I. Guerrero

In order to discover the best location for a Gastropub in the city of Quito in Ecuador, an analysis of the existing pubs, bars, etc. is necessary. With the help of the Foursquare API, I'm going to find out the neighbourhoods with the highest presence of similar venues. By analyzing the Top neighborhoods, I'm going to give a suggestion where this gastropub can be opened in order to secure its success.

For this analysis, I'm going to be using the API of Foursquare in order to extract the venues in the city.
For the data frame of the neighbourhoods I'm going to use a table that can be found in the following URL:
http://www.codigopostalecuador.com/quito-1896

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests
import lxml.html as lh

from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import the data of the Neighborhoods in Quito, Ecuador

In [2]:
#Extract a data frame of the neighborhoods of Quito and their Postal Codes. 
url="http://www.codigopostalecuador.com/quito-1896"

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [4]:

#let’s parse the first row as our header
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))



1:"Código"
2:"Lugar"
3:"Latitud"
4:"Longitud"
5:"&nbsp"


In [5]:
#Since our first row is the header, data is stored on the second row onwards
#for j in range(1,len(tr_elements)):

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=5:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
#Just to be sure, let’s check the length of each column.
[len(C) for (title,C) in col]

[86, 86, 86, 86, 86]

In [7]:
##Add the data to a dictionary and convert it to a data frame
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Código,Lugar,Latitud,Longitud,&nbsp
0,170151,Alangasi,-0.30505,-78.41344,
1,170101,Alfaro (Chimbacalle),-0.23333,-78.51667,
2,170152,Amaguaña,-0.38084,-78.51544,
3,170153,Atahualpa (Chabaspamba),-0.18439,-78.49171,
4,170129,Belisario Quevedo,-0.16563,-78.51045,


In [9]:
len(df)

86

In [10]:
#Change the columns to english and drop the last column since it contains no meaningful information 
df.rename(columns={'Código':'Postal Code','Lugar': 'Neighborhood', 'Latitud':'Latitude','Longitud':'Longitude'}, inplace =True)
df.drop('&nbsp', axis=1, inplace=True)
df.head()

,Postal Code,Neighborhood,Latitude,Longitude
0,170151,Alangasi,-0.30505,-78.41344
1,170101,Alfaro (Chimbacalle),-0.23333,-78.51667
2,170152,Amaguaña,-0.38084,-78.51544
3,170153,Atahualpa (Chabaspamba),-0.18439,-78.49171
4,170129,Belisario Quevedo,-0.16563,-78.51045


In [11]:
#Check if the type of information is correct. In this case the Lon. and Lat. are type 'obejct'. They should be type'float'
df.dtypes

Postal Code     object
Neighborhood    object
Latitude        object
Longitude       object
dtype: object

In [12]:
#Type casting 'object' to 'float'
df['Latitude'] = df['Latitude'].astype('float') 
df['Longitude'] = df['Longitude'].astype('float')
df.dtypes

Postal Code      object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [13]:
#Obtaining the longitude and latitude of Quito
from geopy.geocoders import Nominatim

address = "Quito, Ecuador"

geolocator = Nominatim(user_agent="quito_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Latitude={}, Longitude={}".format(latitude, longitude))

Latitude=-0.2201641, Longitude=-78.5123274


In [14]:
import folium

# create map of Quito using latitude and longitude values
map_quito = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quito)

map_quito

In [15]:
#In the table I found, there was some extra data. Now we have to clean the data frame a little bit
df_clean = df.set_index('Neighborhood')

outsiders = ['Mindo','Puerto Quito','Pedro Vicente Maldonado', 'Pacto','Gualea', 'Nanegalito', 'Nanegal',
            'Chavezpamba','Perucho', 'Puellaro', 'Pintag']
df_clean.drop(outsiders, inplace=True)

In [16]:
df_clean.reset_index(inplace=True)
len(df_clean)

75

In [17]:
# create a second map of Quito using latitude and longitude values
map_quito = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_clean['Latitude'], df_clean['Longitude'], df_clean['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_quito)

map_quito

# Import the data of the venues in Quito and find out which are the top 3 with the highest pressence of Bars, Pubs, Breweries, etc.

In [18]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'F52OIEVBKQK3TDLU1OBLIYE4I2AO22A5XIWTM2Y4XBFTYAMC' 
CLIENT_SECRET = '1XDIO2E2ENSJSOS0TVRQTZLXA1X42X1DUMD5AHTK1EY2GPXJ'
VERSION = '20180605'
#Define the limit of responses and radius of search
LIMIT = 100
radius = 500

In [19]:
#Function to explore and get nearby venues for a data frame of neighborhoods 
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
quito_venues = getNearbyVenues(names=df_clean['Neighborhood'],
                                   latitudes=df_clean['Latitude'],
                                   longitudes=df_clean['Longitude'],
                                  )

Alangasi
Alfaro (Chimbacalle)
Amaguaña
Atahualpa (Chabaspamba)
Belisario Quevedo
Benalcazar
Calacali
Calderon (Carapungo)
Carcelen
Centro Historico
Chaupicruz (La Concepcion)
Checa (Chilpa)
Chilibulo
Chillogallo
Chimbacalle
Cochapamba
Comite Del Pueblo
Condado
Conocoto
Cotocollao
Cumbaya
El Batan
El Beaterio
El Inca
El Quinche
El Salvador
Eloy Alfaro
Gonzalez Suarez
Guamani
Guangopolo
Guapulo
Guayllabamba
Iñaquito
Itchimbia
Jipijapa
Kennedy
La Argelia
La Concepción
La Ecuatoriana
La Ferroviaria
La Floresta
La Libertad
La Magdalena
La Mena
La Merced
La Vicentina
Las Cuadras
Llano Chico
Lloa
Mariscal Sucre
Nayón
Nono
Pifo
Pomasqui
Ponceano
Puembo
Puengasi
Quito
Quitumbe
Rumipamba
San Antonio
San Bartolo
San Blas
San Isidro Del Inca
San Jose De Minas
San Marcos
San Roque
San Sebastian
Santa Barbara
Santa Prisca
Tababela
Tumbaco
Villaflora
Yaruquì
Zàmbiza


In [21]:
print(quito_venues.shape)


(1191, 7)


In [22]:
condition1 = quito_venues['Venue Category'] == 'Bar'
condition2 = quito_venues['Venue Category'] == 'Gastropub'
condition3 = quito_venues['Venue Category'] == 'Brewery'
condition4 = quito_venues['Venue Category'] == 'Hotel Bar'
condition5 = quito_venues['Venue Category'] == 'Lounge'
condition6 = quito_venues['Venue Category'] == 'Pub'
condition7 = quito_venues['Venue Category'] == 'Cocktail'

df_pub = quito_venues[(condition1) | (condition2) | (condition3) | (condition4) | (condition5) | (condition6) | (condition7)]

df_pub.reset_index(drop = True, inplace= True)
df_pub.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Atahualpa (Chabaspamba),-0.18439,-78.49171,Western,-0.186218,-78.489056,Bar
1,Benalcazar,-0.18262,-78.48122,Vertigo Rooftop,-0.184088,-78.480879,Hotel Bar
2,Benalcazar,-0.18262,-78.48122,VIVA Cerveza! Gastropub & Beer Store - LA CARO...,-0.185072,-78.480299,Gastropub
3,Benalcazar,-0.18262,-78.48122,Alitasd'lCadillac Quito,-0.185255,-78.477422,Gastropub
4,Benalcazar,-0.18262,-78.48122,El Estanco,-0.179352,-78.486927,Bar


In [23]:
#Analyze Each Neighborhood

# one hot encoding
pubs_onehot = pd.get_dummies(df_pub[['Venue Category']], prefix="", prefix_sep="")
pubs_onehot

# add neighborhood column back to dataframe
pubs_onehot =pubs_onehot.join(df_pub['Neighborhood'],lsuffix='_caller')

# move neighborhood column to the first column
fixed_columns = [pubs_onehot.columns[-1]] + list(pubs_onehot.columns[:-1])
pubs_onehot = pubs_onehot[fixed_columns]

#print the data frame
pubs_onehot.head()

,Neighborhood,Bar,Brewery,Gastropub,Hotel Bar,Lounge,Pub
0,Atahualpa (Chabaspamba),1,0,0,0,0,0
1,Benalcazar,0,0,0,1,0,0
2,Benalcazar,0,0,1,0,0,0
3,Benalcazar,0,0,1,0,0,0
4,Benalcazar,1,0,0,0,0,0


In [24]:
#Add a total column and group the venues by neighborhood
pubs_onehot['Total'] = pubs_onehot.sum(axis=1)  

pubs_grouped = pubs_onehot.groupby('Neighborhood').sum().reset_index()
print(pubs_grouped.shape)
pubs_grouped.head()

(16, 8)


,Neighborhood,Bar,Brewery,Gastropub,Hotel Bar,Lounge,Pub,Total
0,Atahualpa (Chabaspamba),1,0,0,0,0,0,1
1,Benalcazar,1,0,2,1,0,0,4
2,Chaupicruz (La Concepcion),1,0,0,1,0,0,2
3,Condado,0,0,0,0,1,0,1
4,Cumbaya,2,1,0,0,0,0,3


In [25]:
#Top 3 Neighborhoods with highest presence of bars, breweries, etc.
top_3 = pubs_grouped.nlargest(3, 'Total').reset_index(drop=True)
top_3

,Neighborhood,Bar,Brewery,Gastropub,Hotel Bar,Lounge,Pub,Total
0,La Floresta,1,3,0,0,0,2,6
1,Guapulo,2,1,0,0,1,1,5
2,Mariscal Sucre,2,1,1,0,1,0,5


In [26]:
N1 = df_pub['Neighborhood'] == top_3['Neighborhood'].loc[0]
N2 = df_pub['Neighborhood'] == top_3['Neighborhood'].loc[1]
N3 = df_pub['Neighborhood'] == top_3['Neighborhood'].loc[2]
pubs_top_3 = df_pub[(N1)|(N2)|(N3)]

print(pubs_top_3.shape)
pubs_top_3.reset_index(drop=True, inplace = True) 
pubs_top_3.head()

(16, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Guapulo,-0.20474,-78.47835,Strawberry Fields,-0.201690,-78.480360,Lounge
1,Guapulo,-0.20474,-78.47835,Western,-0.206730,-78.484311,Bar
2,Guapulo,-0.20474,-78.47835,Grima,-0.206623,-78.483755,Pub
3,Guapulo,-0.20474,-78.47835,Santa Rosa Cerveza Artesanal,-0.206641,-78.484646,Brewery
4,Guapulo,-0.20474,-78.47835,Brasa Viva,-0.199323,-78.481562,Bar


# Analysis of the top 3 Neighborhoods in Quito

In [27]:
#Create a data frame with only the venues of the top 3 neighborhoods
N1 = quito_venues['Neighborhood'] == top_3['Neighborhood'].loc[0]
N2 = quito_venues['Neighborhood'] == top_3['Neighborhood'].loc[1]
N3 = quito_venues['Neighborhood'] == top_3['Neighborhood'].loc[2]
venues_top = quito_venues[(N1)|(N2)|(N3)]

print(venues_top.shape)
venues_top.head().reset_index(drop=True) 


(180, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Guapulo,-0.20474,-78.47835,Segundo Muelle,-0.204057,-78.481530,Peruvian Restaurant
1,Guapulo,-0.20474,-78.47835,Lucía Pie House & Grill,-0.202972,-78.480830,Southern / Soul Food Restaurant
2,Guapulo,-0.20474,-78.47835,Warmi Artesanalmente Gourmet,-0.203046,-78.478956,Restaurant
3,Guapulo,-0.20474,-78.47835,Mirador De Guápulo,-0.201517,-78.479250,Scenic Lookout
4,Guapulo,-0.20474,-78.47835,Noe Sushi Bar,-0.204338,-78.482126,Sushi Restaurant


In [28]:
# one hot encoding
top_onehot = pd.get_dummies(venues_top[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
top_onehot =top_onehot.join(venues_top['Neighborhood'],lsuffix='_caller')

# move neighborhood column to the first column
fixed_columns = [top_onehot.columns[-1]] + list(top_onehot.columns[:-1])
top_onehot = top_onehot[fixed_columns]

top_onehot.shape

(180, 72)

In [29]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
top_grouped = top_onehot.groupby('Neighborhood').mean().reset_index()
print(top_grouped.shape)
top_grouped 

(3, 72)


,Neighborhood,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bar,Bike Rental / Bike Share,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comedy Club,Dessert Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Greek Restaurant,Gym / Fitness Center,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Mexican Restaurant,Middle Eastern Restaurant,Motel,Mountain,Music Venue,Nightclub,Park,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Supermarket,Sushi Restaurant,Taco Place,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Guapulo,0.000000,0.000000,0.016393,0.000000,0.016393,0.016393,0.000000,0.032787,0.000000,0.000000,0.016393,0.016393,0.016393,0.049180,0.000000,0.000000,0.016393,0.049180,0.032787,0.016393,0.032787,0.000000,0.016393,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.049180,0.016393,0.000000,0.016393,0.049180,0.016393,0.016393,0.032787,0.016393,0.016393,0.00000,0.000000,0.000000,0.000000,0.032787,0.000000,0.032787,0.016393,0.049180,0.000000,0.016393,0.049180,0.000000,0.016393,0.049180,0.000000,0.000000,0.016393,0.000000,0.000000,0.049180,0.016393,0.000000,0.000000,0.016393,0.032787
1,La Floresta,0.000000,0.000000,0.000000,0.000000,0.017241,0.034483,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.051724,0.034483,0.000000,0.000000,0.017241,0.034483,0.034483,0.000000,0.034483,0.000000,0.017241,0.000000,0.017241,0.000000,0.017241,0.017241,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.017241,0.068966,0.034483,0.017241,0.017241,0.000000,0.034483,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.034483,0.017241,0.017241,0.000000,0.034483,0.017241,0.017241,0.017241,0.017241,0.017241,0.051724,0.017241,0.000000,0.000000,0.017241,0.034483
2,Mariscal Sucre,0.016393,0.016393,0.000000,0.016393,0.032787,0.016393,0.000000,0.032787,0.000000,0.016393,0.000000,0.000000,0.016393,0.000000,0.016393,0.016393,0.000000,0.049180,0.016393,0.000000,0.049180,0.016393,0.000000,0.016393,0.000000,0.016393,0.032787,0.016393,0.032787,0.000000,0.016393,0.016393,0.016393,0.016393,0.000000,0.065574,0.000000,0.016393,0.000000,0.016393,0.000000,0.000000,0.032787,0.016393,0.016393,0.04918,0.016393,0.016393,0.016393,0.016393,0.016393,0.000000,0.016393,0.032787,0.016393,0.000000,0.032787,0.016393,0.000000,0.032787,0.000000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,0.016393,0.000000,0.000000


In [30]:
#print each neighborhood along with the top 5 most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = top_grouped ['Neighborhood']

for ind in np.arange(top_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(top_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Guapulo,Hotel,Restaurant,Burger Joint,Café,Sushi Restaurant
1,La Floresta,Italian Restaurant,Brewery,Sushi Restaurant,Wings Joint,Pub
2,Mariscal Sucre,Hotel,Café,Middle Eastern Restaurant,Coffee Shop,Asian Restaurant


In [32]:
#change latitude and longitude values for visualization purposes 
latitude = -0.206589
longitude = -78.488071
# create a second map of Quito using latitude and longitude values
map_quito = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, Neighborhood in zip(pubs_top_3['Venue Latitude'], pubs_top_3['Venue Longitude'], pubs_top_3['Venue'],pubs_top_3['Neighborhood']):
    if (Neighborhood == top_3['Neighborhood'].loc[0]):
        my_color = 'blue'
    elif (Neighborhood == top_3['Neighborhood'].loc[1]):
        my_color = 'green'
    else:
        my_color = 'black'
    
    label = venue
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=my_color,
        fill=True,
        fill_color=my_color,
        fill_opacity=0.5,
        parse_html=False).add_to(map_quito)

map_quito

# Conclusion

Based on the data, we can assume the following:
   * The top 3 neighborhoods with the highest presence of bars, pubs, etc., are "Guapulo", "La floresta" and "Mariscal Sucre".
   * The top 3 neighborhoods are really close to each other and they are situated in the center of the city.
   * All three locations appear to be very commercial and have many venues for leisure.
   * There is no clear winner.The gastro pub can be opened in any of the three neighbourhoods, given the proximity between them and their similarities.